In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

# Download the list of S&P500 stocks from Wikipedia
sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_df = pd.read_html(sp500_url)[0]
sp500_symbols = sp500_df["Symbol"].tolist()

# Download daily historical stock prices for S&P500 stocks
start_date = "2010-01-01"
end_date = "2022-12-31"

In [ ]:
for symbol in sp500_symbols:
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data.to_csv(f"{symbol}.csv")


In [ ]:
# Keep the last observation for each stock/month
stock_data_monthly = {}

for symbol in sp500_symbols:
    stock_data = pd.read_csv(f"{symbol}.csv")
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    stock_data = stock_data.set_index('Date')
    stock_data_monthly[symbol] = stock_data.resample("M").last()

In [ ]:
# Calculate monthly returns and forward returns
for symbol, data in stock_data_monthly.items():
    data["pct_change"] = data["Close"].pct_change()
    data["forward_return"] = data["pct_change"].shift(-1)

In [ ]:
# Restrict the observations to the time period 2010/1 - 2022/12
for symbol, data in stock_data_monthly.items():
    data = data.loc[data.index >= "2010-01-01"]
    data = data.loc[data.index <= "2022-12-31"]


In [ ]:
# Read in the file "consolidated_quarterly_balance_sheets.csv"
balance_sheet_data = pd.read_csv("consolidated_quarterly_balance_sheets.csv")

# Calculate the current ratio for each observation
balance_sheet_data["current_ratio"] = balance_sheet_data["AssetsCurrent"] / balance_sheet_data["LiabilitiesCurrent"]


In [ ]:
balance_sheet_data = pd.DataFrame(balance_sheet_data)
stock_data_monthly = pd.Series(stock_data_monthly, index=[0])
stock_data_merged = pd.concat([stock_data_monthly, balance_sheet_data], axis=1)
# print(stock_data_merged.columns)

In [ ]:
# Fill in the missing observations of the current ratio with its last known value for the same firm
stock_data_merged["current_ratio"] = stock_data_merged["current_ratio"].fillna(method="ffill")

In [ ]:
# Every month, sort the stocks into five bins based on their current ratios. Form an equally-weighted portfolio for each bin
stock_data_monthly = pd.DataFrame(stock_data)

# j = 0
for date in stock_data_merged.index:
    stock_data_current = stock_data_merged.loc[date]
    # stock_data_current = stock_data_current.sort_values("current_ratio")

    portfolio_bins = np.array_split(stock_data_current, 5)
    # print(portfolio_bins)

    # print(list(portfolio_bins[0].values)[6])
    # print(portfolio_bins[0])
    for i in range(5):
        portfolio_bin = portfolio_bins[i]
        portfolio_bin["weight"] = 1 / len(portfolio_bin)
        portfolio_bin["return"] =list(portfolio_bins[0].values)[6].mean()

    # j += 1
    # print(j)

In [ ]:
# Calculate summary statistics of returns for each portfolio
print(stock_data_merged.keys())

portfolio_returns_short = stock_data_merged.groupby("quarter").mean()

portfolio_returns_long = stock_data_merged.groupby("year").mean()

if portfolio_returns_long.describe().mean()["Assets"] > portfolio_returns_short.describe().mean()["Assets"]:
    print("long-short portfolio yields positive returns on average")

else:
    print("long-short portfolio yields negative returns on average")
